# EEE3096S Practical 4 
## Date: 8 October 2021
## Authors:
### Angus Longmore (LNGANG002) and Malcom Baatjies (BTJMAL001)

![Diagram](new_circuit.png)

## Validation and Testing

The first tests for the ldr involved measuring the readings at ambient, then covering the light, observing the readings decrease, shining a torch on the light and observing the readings increase. 
To test for the Thermistor, the ADC readings were converted to Celcius, then compared to the value of a mercury thermometer. The readings were seperated by a degree and half.

The Pi was then restarted and the tests redone to validate the results. We obtained very similar readings for ambient temperature and light, the increase/decreases were also consistent with our initial testing.

The image below is a sample of some of the testing. The sections outlined in red are ambient readings.The section in blue is the light sensor being covered. The section in green is the thermistor being held by fingers (increasing the temperature), notice the ramp up as the thermistor warms. You'll also notice during this section the ldr readings drop, this is due to the ldr being close to the thermistor, and the hand of the operator blocking the light to the ldr.
The pink section is when the thermistor has been released and is returning to ambient slowly.

We will demo our program in person next week, as indicated we can do on the discord server, so no video is linked.

![Testing](Practical-4-example-marked.png)

In [ ]:
import busio
import digitalio
import board
import threading
import RPi.GPIO as GPIO
from time import time, sleep
import adafruit_mcp3xxx.mcp3008 as MCP
from adafruit_mcp3xxx.analog_in import AnalogIn

def print_sensor_vals(start_time, temp, ldr):
    print(f"{round(time()-start_time)}s\t{temp.value}\t \t{round(((temp.voltage-0.5)/0.01), 2)}\t{round(ldr.voltage, 2)}")


def cycle_sample_time(channel):
    global current_sampling_time_index, sampling_times
    print(f"Changing from {sampling_times[current_sampling_time_index]}s to {sampling_times[(current_sampling_time_index+1)%3]}s after the next sample")
    current_sampling_time_index = (current_sampling_time_index +1) %3

sampling_times = [10, 5, 1]
current_sampling_time_index = 0

#create the spi bus
spi = busio.SPI(clock=board.SCK, MISO=board.MISO, MOSI=board.MOSI)

# create the cs (chip select)
cs = digitalio.DigitalInOut(board.D5)


# create the mcp object
mcp = MCP.MCP3008(spi, cs)

# Add button callback to change sampling rate
GPIO.setup(17, GPIO.IN, pull_up_down=GPIO.PUD_UP)
GPIO.add_event_detect(17, GPIO.FALLING, callback=cycle_sample_time, bouncetime=200)
    
start_time = time()
ldr = AnalogIn(mcp, MCP.P2)
temp = AnalogIn(mcp, MCP.P1)
print("Runtime\tTemp Reading\tTemp\tLight Reading")
while True:
    x = threading.Thread(target=print_sensor_vals, args=(start_time, temp, ldr))
    x.start()
    x.join()
    wait_time = sampling_times[current_sampling_time_index]
    sleep(wait_time)